# Lab Control

In this tutorial, we take three different pieces of scientific equipment and use this 
[Jupyter](https://jupyter.org/) Notebook to control the devices using 
[Python](https://python.org/).

The devices which we are controlling in this tutorial are:

* [Keysight 33512B Waveform Generator](https://www.keysight.com/au/en/assets/7018-05928/data-sheets/5992-2572.pdf)
* [Rigol DS1074 Oscilloscope](https://www.rigolna.com/ds1z/)
* [Brüel & Kjær Type 3050-B-6](https://www.bksv.com/en/products/data-acquisition-systems-and-hardware/LAN-XI-data-acquisition-hardware/modules/type-3050)

The first step in remotely controlling these devices is to connect them to a computer via either USB or Ethernet.

## Physical setup of the devices

### Front Panels

The three devices are connected to each other using [BNC](https://en.wikipedia.org/wiki/BNC_connector) connectors and cables.

You will require 4 BNC cables and 2 BNC Tee Adapters.

![Device configuration (front)](figures/Device_configuration_front.png)

### Rear panels (Ethernet)

The three devices can be connected to the network using Ethernet cables and an unmanaged switch.

You will require 4 Ethernet cables and an unmanaged switch.

![Device configuration (Ethernet)](figures/Device_configuration_ethernet.png)

The devices can either be connected directly to the computer via the switch, or the switch can be connected to a router with WiFi access for the computer.

### Rear panels (USB)

The signal generator and the oscilloscope can be connected to the computer via USB.
The DAQ will still require Ethernet connection.

You will require 1 Ethernet cable and 2 USB cables.

![Device configuration (USB)](figures/Device_configuration_usb.png)


## Connecting VISA compatible devices via USB

Connecting a device via USB does not require any configuration of the device, but will generally require configuration on the computer to which you are attaching the device. 
This configuration is different for different operating systems. Here we run through the configuration for Linux and Windows 10.

### Linux

Start by connecting the device to your computer via USB cable, and turning on the device.
Then get a list of currently connected USB devices:

In [ ]:
usb_devices = subprocess.check_output('lsusb').decode('UTF-8')
print(usb_devices)

Hopefully the device is contained in the list. In my case, the device is the 4th on the list, and from this I can obtain the Vendor ID and the Product ID:

In [ ]:
#Keysight 3500B Series
idVendor = '0957'
idProduct = '2807'

We now need to add the device to the list of devices that we can utilise (NB: this requires root access).

The line that we need to add is:

In [ ]:
import getpass
new_usb_rule = (f'SUBSYSTEMS=="usb", ACTION=="add", ' +
                f'ATTRS{{idVendor}}=="{idVendor}", ' +
                f'ATTRS{{idProduct}}=="{idProduct}", ' +
                f'GROUP="{getpass.getuser()}", MODE="0660"'
               )
print(new_usb_rule)

Check that the rule is not already there:

In [ ]:
!cat /etc/udev/rules.d/usbtmc.rules

If it is not already there, then add it by running the following commands in a terminal window:

In [ ]:
print(f"echo '{new_usb_rule}' | sudo tee -a /etc/udev/rules.d/usbtmc.rules")
print('sudo udevadm control --reload-rules && udevadm trigger')

In [ ]:
!pip install libusb1

In [ ]:
import usb1
with usb1.USBContext() as context:
    handle = context.openByVendorIDAndProductID(
        idVendor,
        idProduct,
        skip_on_error=True,
    )
    if handle is None:
        # Device not present, or user is not allowed to access device.
        print("Device not present")
    with handle.claimInterface(INTERFACE):
        # Do stuff with endpoints on claimed interface.
        #print(Agilent.ask("*IDN?"))
        pass

In [ ]:
import usbtmc
import usb.core
import usb.backend.libusb1
Agilent = usbtmc.Instrument(idVendor,idProduct)
print(Agilent.ask("*IDN?"))

In [ ]:
usbtmc?

In [ ]:
#Rigol DS1074Z
idVendor = 
idProduct = 

### Windows 10

In [ ]:
!pip install pyvisa

Install [NI VISA](https://pyvisa.readthedocs.io/en/latest/faq/getting_nivisa.html#faq-getting-nivisa)

In [ ]:
!lsusb

In [ ]:
import pyvisa
rm = pyvisa.ResourceManager()

print(rm.list_resources())

Agilent = rm.open_resource(rm.list_resources()[0])
print(Agilent.query("*IDN?"))

## Connecting vxi11 compatible devices via Ethernet

Connecting devices via Ethernet is generally easier than connecting via USB. There is no need to load any drivers or run any root or admin commands, however the computer that you are using and the devices themselves all need to be on the same WAN.



In [ ]:
!pip install python-vxi11

In [ ]:
keysight_ip = '192.168.0.55'

In [ ]:
import vxi11
import time
Agilent = vxi11.Instrument(keysight_ip)
print(Agilent.ask("*IDN?"))

In [ ]:
rigol_ip = '192.168.0.60'

In [ ]:
Rigol = vxi11.Instrument(rigol_ip)
print(Rigol.ask("*IDN?"))

## Interacting with a Brüel & Kjær Type 3050-B-6 using Python

To communicate with a stand-alone [Brüel & Kjær Type 3050-B-6](https://www.bksv.com/en/products/data-acquisition-systems-and-hardware/LAN-XI-data-acquisition-hardware/modules/type-3050), it is necessary to have a [Notar™ BZ-7848-A (LAN-XI stand-alone recorder license)](https://www.bksv.com/en/products/data-acquisition-systems-and-hardware/general-purpose-analyzer-system/lan-xi-notar), which allows you to interact with the device via a browser, utilising the Ethernet port at the back of the device. 

The details of how to interact with the BnK have been worked out in the [PyBnK](https://github.com/uwasystemhealth/PyBnK) respository, here we simply need to import the module, then use the BnK device.

In [ ]:
bnk_ip = '192.168.0.29'